In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import pickle
from datetime import datetime


import pandas as pd
import numpy as np
from datetime import datetime
from binance.client import Client
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor
from datetime import datetime
import btalib

pd.set_option('display.max_rows', 500)


import logging

# Gets or creates a logger
logger = logging.getLogger(__name__)  

# set log level
logger.setLevel(logging.INFO)

# define file handler and set formatter
file_handler = logging.FileHandler('logfile.log')
formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

In [2]:
def f_get_coinlist():
    coin_list_df = pd.read_csv("crypto_lu.csv")
    coin_list_df["Symbolusd"] =  coin_list_df["Symbol"].apply(lambda x: x + "USDT" )
    coin_list_df["Keywords"]  = coin_list_df.apply(lambda row: row["Name"].lower() + "," + 
                                   row["Symbol"].lower() + "," + 
                                   row["Symbolusd"].lower() + "," + 
                                   row["Keywords"].lower()  , axis = 1)


    coin_list_df = coin_list_df.head(25)

    coin_list = list(coin_list_df.Symbolusd)
    return coin_list


def f_get_binance_client():
    api_key = os.environ.get('BINANCE_API')
    api_secret = os.environ.get('BINANCE_SECRET')
    return Client(api_key, api_secret)


def f_load_allcoins(coin_list = None):
    
    client = f_get_binance_client()
    coin_list = f_get_coinlist()
    dict_all = {}
    for coin in coin_list:  
        dict_coin = {}    
        logger.info('Initial load {x}'.format(x=coin))
        try:
            klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_2HOUR, "20 day ago UTC")
        except Exception as e:
            print(str(e))
            logger.error('Initial load {x} FAILED'.format(x=coin))
            continue
        for x in klines:
            ts = int(x[0]) / 1000
            dt = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            dict_coin.update({dt:[x[0], x[4], x[5], x[8]]}) #date, price, volume, numoftrades    
        dict_all.update({coin:dict_coin})

    return dict_all



# fetch 1 minute klines for the last day up until now
def f_update_lastprice(dict_all=None):
    
    client = f_get_binance_client()
    for coin in list(dict_all.keys()):  
        dict_coin = {}
        logger.info('try to update coin {x}'.format(x=coin))
        
        try:
            klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_1MINUTE, "1 minute ago UTC")
        except Exception as e:
            print(str(e))
            logger.error('UPDATE list of coin: {x} FAILED, seems no info'.format(x=coin))
            ##Make warning to say that such coin data is not up-to-date!!
            continue        
        dict_coin = dict_all.get(coin)
        for x in klines:
            ts = int(x[0]) / 1000
            dt = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            dict_coin.update({dt:[x[0], x[4], x[5], x[8]]}) #date, price, volume, numoftrades    
        dict_all.update({coin:dict_coin})

    return dict_all



def f_get_all_price():
    dict_all_coins = f_load_allcoins(coin_list = None)
    dict_all_coins = f_update_lastprice(dict_all=dict_all_coins)
    
    df_all_coins = pd.DataFrame(columns= ["coin", "dt", "price", "vol", "tradefreq"])
    for coin in dict_all_coins.keys():
        dict_coin = dict_all_coins.get(coin)
        listx = []
        for dt in dict_coin.keys():
            listx.append([coin, dt, dict_coin.get(dt)[1], dict_coin.get(dt)[2], dict_coin.get(dt)[3]])
        df_all_coins = df_all_coins.append(pd.DataFrame(data = listx,
                                         columns= ["coin", "dt", "price", "vol", "tradefreq"]
                                        )
                           )
    df_all_coins.dt = df_all_coins.dt.apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')  )
    df_all_coins['last_time'] = 0
    for coin in df_all_coins.coin.unique():
        max_dt = np.max(df_all_coins.loc[df_all_coins.coin == coin, 'dt'])
        df_all_coins.loc[(df_all_coins.coin == coin) & (df_all_coins.dt == max_dt ) , 'last_time'] = 1

    df_all_coins.set_index("dt", drop=True, inplace = True )
    df_all_coins.sort_index(inplace=True)
    df_all_coins["is_day_end"] = 0
    df_all_coins.loc[[True if x.strftime("%H:%M:%S") == '00:00:00' else False for x in list(df_all_coins.index)], "is_day_end"] = 1  
    df_all_coins.price = df_all_coins.price.astype(float)
    df_all_coins.tradefreq= df_all_coins.tradefreq.astype(float)
    df_all_coins.vol= df_all_coins.vol.astype(float)
    
    
    return df_all_coins
    


def f_calc_indicators():

    df_all_coins = f_get_all_price()
    
    for coin in df_all_coins.coin.unique():
        index_t = (df_all_coins.coin == coin)
        df_all_coins.loc[index_t, 'ma12'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=12).df.values.tolist()
        df_all_coins.loc[index_t, 'ma24'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=24).df.values.tolist()
        df_all_coins.loc[index_t, 'ma48'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=48).df.values.tolist()
        #df_all_coins.loc[index_t, 'ma148'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=96).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi14'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=14).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi28'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=28).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi42'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=42).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi48'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=48).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi72'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=72).df.values.tolist()
        df_all_coins.loc[index_t, ["macd12", "signal12", "hist12"] ] =  \
                        btalib.macd(df_all_coins.loc[index_t, 'price'], pfast=12, pslow=24, psignal=9).df.values.tolist()
        df_all_coins.loc[index_t, ["macd6", "signal6", "hist6"] ] =  \
                        btalib.macd(df_all_coins.loc[index_t, 'price'], pfast=6, pslow=12, psignal=4).df.values.tolist()

    df_all_coins['ma12_24_diff'] = df_all_coins['ma12'] - df_all_coins['ma24']
    df_all_coins['macd12_ratio'] = df_all_coins['macd12'] / df_all_coins['price']
    df_all_coins['signal12_ratio'] = df_all_coins['signal12'] / df_all_coins['price']
    
    
    return df_all_coins


def f_get_last_minute_rows():
    df = f_calc_indicators()
    df.reset_index(inplace=True)
    df.loc[:, "rnk_dt"] =  df.groupby("coin")["dt"].rank("dense", ascending=False)
    return df.loc[df.rnk_dt == 1, ["dt", "coin", "price", "rsi14", "rsi28", "macd12_ratio", "signal12_ratio"]]
    
    


In [3]:
f_get_last_minute_rows()
#dict_all_coins = f_load_allcoins(coin_list = coin_list)
#dict_all_coins = f_update_lastprice(dict_all=dict_all_coins)


APIError(code=-1121): Invalid symbol.
APIError(code=-1121): Invalid symbol.
APIError(code=-1121): Invalid symbol.


,dt,coin,price,rsi14,rsi28,macd12_ratio,signal12_ratio
5228,2021-05-04 20:00:00,NEOUSDT,99.98500,44.765058,48.740628,-0.013275,-0.006003
5229,2021-05-04 20:00:00,LUNAUSDT,16.41600,40.800113,45.136495,-0.012771,-0.006059
5231,2021-05-04 20:00:00,XMRUSDT,397.10000,44.306628,45.862309,-0.011417,-0.010233
5236,2021-05-04 21:17:00,BTCUSDT,54558.02000,36.548281,43.032839,-0.012116,-0.008560
5237,2021-05-04 21:17:00,ETHUSDT,3380.40000,58.050297,60.580935,0.023700,0.026267
5238,2021-05-04 21:17:00,TRXUSDT,0.12365,42.609154,46.213879,-0.013121,-0.008883
5239,2021-05-04 21:17:00,BNBUSDT,628.28000,45.684873,50.437265,-0.002380,0.004277
5240,2021-05-04 21:17:00,XRPUSDT,1.46050,44.858817,46.883800,-0.021876,-0.019863
5241,2021-05-04 21:17:00,FILUSDT,146.00000,29.245967,36.969132,-0.023843,-0.017124
5242,2021-05-04 21:17:00,ADAUSDT,1.31210,47.194505,48.516313,-0.006584,-0.005188


In [53]:
df.loc[df.coin == 'DOGEUSDT', :]

,coin,price,vol,tradefreq,last_time,is_day_end,ma12,ma24,ma48,rsi14,...,rsi48,rsi72,macd12,signal12,hist12,macd6,signal6,hist6,ma12_24_diff,macd12_ratio
dt,,,,,,,,,,,,,,,,,,,,,
2021-04-14 22:00:00,DOGEUSDT,0.121433,1.813267e+09,239993.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-15 00:00:00,DOGEUSDT,0.127398,1.768507e+09,258000.0,0,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-15 02:00:00,DOGEUSDT,0.128802,7.957302e+08,132659.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-15 04:00:00,DOGEUSDT,0.135869,2.741354e+09,431978.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-15 06:00:00,DOGEUSDT,0.136280,1.367447e+09,212735.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-15 08:00:00,DOGEUSDT,0.132299,1.042996e+09,161442.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-15 10:00:00,DOGEUSDT,0.128897,9.236121e+08,140561.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-15 12:00:00,DOGEUSDT,0.133113,7.411907e+08,117914.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-15 14:00:00,DOGEUSDT,0.133100,6.102482e+08,98625.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df.reset_index(inplace=True)
df.loc[:, "rnk_dt"] =  df.groupby("coin")["dt"].rank("dense", ascending=False)
df.loc[df.rnk_dt == 1, ["dt", "coin", "price", "rsi14", "rsi28", "macd12", "signal12"]]

,dt,coin,price,rsi14,rsi28,macd12,signal12
5236,2021-05-04 21:09:00,LINKUSDT,47.24600,71.406694,66.731987,1.522579,0.932456
5237,2021-05-04 21:09:00,LUNAUSDT,16.38400,40.449545,44.922951,-0.226793,-0.125292
5238,2021-05-04 21:09:00,BTCUSDT,54549.86000,36.333041,42.959232,-663.959595,-468.086277
5239,2021-05-04 21:09:00,ETHUSDT,3373.54000,57.656315,60.304093,79.041972,88.458675
5240,2021-05-04 21:09:00,BNBUSDT,626.98000,45.238213,50.153013,-1.686877,2.628773
5241,2021-05-04 21:09:00,XRPUSDT,1.44000,41.807664,45.288184,-0.034415,-0.029702
5242,2021-05-04 21:09:00,ADAUSDT,1.30700,46.033563,47.900198,-0.009355,-0.007021
5243,2021-05-04 21:09:00,DOGEUSDT,0.53871,66.393694,67.393474,0.040119,0.033347
5244,2021-05-04 21:09:00,DOTUSDT,35.61600,43.400098,47.476045,-0.244143,-0.126403
5245,2021-05-04 21:09:00,UNIUSDT,43.61200,57.048563,55.588388,0.234024,0.106793


In [40]:
df

,dt,coin,price,vol,tradefreq,last_time,is_day_end,ma12,ma24,ma48,...,rsi72,macd12,signal12,hist12,macd6,signal6,hist6,ma12_24_diff,macd12_ratio,rnk_dt
0,2021-04-14 22:00:00,USDTRYUSDT,0.008732,55659.43000,29722.0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238.0
1,2021-04-14 22:00:00,ETHUSDT,0.008732,55659.43000,29722.0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.0
2,2021-04-14 22:00:00,BUSDUSDT,0.008732,55659.43000,29722.0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.0
3,2021-04-14 22:00:00,BNBUSDT,0.008732,55659.43000,29722.0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.0
4,2021-04-14 22:00:00,LUNAUSDT,0.008732,55659.43000,29722.0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5966,2021-05-04 20:44:00,XMRUSDT,395.390000,2.33262,6.0,1,0,32.959789,16.485516,8.248174,...,99.999172,29.197287,5.839577,23.357710,52.137828,20.855146,31.282681,16.474273,-366.192713,1.0
5967,2021-05-04 20:44:00,XLMUSDT,0.513470,2988.20000,4.0,1,0,0.053411,0.032327,0.021580,...,99.356883,0.037174,0.007554,0.029619,0.066197,0.026494,0.039703,0.021084,-0.476296,1.0
5968,2021-05-04 20:44:00,LUNAUSDT,16.378000,15.26400,1.0,1,0,1.375455,0.693349,0.352091,...,99.980010,1.208708,0.241861,0.966847,2.158223,0.863305,1.294919,0.682106,-15.169292,1.0
5969,2021-05-04 20:44:00,TRXUSDT,0.122810,58168.90000,16.0,1,0,0.020856,0.016049,0.013441,...,97.240987,0.008325,0.001785,0.006540,0.014682,0.005888,0.008794,0.004807,-0.114485,1.0


In [34]:
df

,dt,coin,price,vol,tradefreq,last_time,is_day_end,ma12,ma24,ma48,...,macd12,signal12,hist12,macd6,signal6,hist6,ma12_24_diff,macd12_ratio,rank_dt,rnk_dt
0,2021-04-14 22:00:00,USDTRYUSDT,0.008732,5.565943e+04,29722.0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238.0
1,2021-04-14 22:00:00,ETHUSDT,0.008732,5.565943e+04,29722.0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.0
2,2021-04-14 22:00:00,BUSDUSDT,0.008732,5.565943e+04,29722.0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.0
3,2021-04-14 22:00:00,BNBUSDT,0.008732,5.565943e+04,29722.0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.0
4,2021-04-14 22:00:00,LUNAUSDT,0.008732,5.565943e+04,29722.0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5967,2021-05-04 20:34:00,LTCUSDT,303.790000,1.034610e+03,324.0,1,0,25.326455,12.668849,6.339841,...,22.432979,4.486715,17.946264,40.058706,16.023498,24.035209,12.657606,-281.357021,NaN,1.0
5968,2021-05-04 20:34:00,BUSDUSDT,0.999000,1.383298e+06,265.0,1,0,0.093872,0.052557,0.031695,...,0.073028,0.014725,0.058303,0.130223,0.052104,0.078119,0.041314,-0.925972,NaN,1.0
5969,2021-05-04 20:34:00,VETUSDT,0.193280,9.932553e+05,342.0,1,0,0.026728,0.018986,0.014909,...,0.013529,0.002825,0.010703,0.023974,0.009605,0.014369,0.007743,-0.179751,NaN,1.0
5970,2021-05-04 20:34:00,BNBUSDT,621.410000,7.776443e+02,516.0,1,0,51.794788,25.903015,12.956924,...,45.887994,9.177718,36.710276,81.942662,32.777080,49.165582,25.891773,-575.522006,NaN,1.0


In [29]:
df

,coin,price,vol,tradefreq,last_time,is_day_end,ma12,ma24,ma48,rsi14,...,rsi72,macd12,signal12,hist12,macd6,signal6,hist6,ma12_24_diff,macd12_ratio,rank_dt
dt,,,,,,,,,,,,,,,,,,,,,
2021-04-14 22:00:00,USDTRYUSDT,0.008732,5.565943e+04,29722.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-14 22:00:00,ETHUSDT,0.008732,5.565943e+04,29722.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-14 22:00:00,BUSDUSDT,0.008732,5.565943e+04,29722.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-14 22:00:00,BNBUSDT,0.008732,5.565943e+04,29722.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-14 22:00:00,LUNAUSDT,0.008732,5.565943e+04,29722.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-04 20:34:00,LTCUSDT,303.790000,1.034610e+03,324.0,1,0,25.326455,12.668849,6.339841,99.999796,...,99.998921,22.432979,4.486715,17.946264,40.058706,16.023498,24.035209,12.657606,-281.357021,NaN
2021-05-04 20:34:00,BUSDUSDT,0.999000,1.383298e+06,265.0,1,0,0.093872,0.052557,0.031695,99.937387,...,99.670727,0.073028,0.014725,0.058303,0.130223,0.052104,0.078119,0.041314,-0.925972,NaN
2021-05-04 20:34:00,VETUSDT,0.193280,9.932553e+05,342.0,1,0,0.026728,0.018986,0.014909,99.660910,...,98.267561,0.013529,0.002825,0.010703,0.023974,0.009605,0.014369,0.007743,-0.179751,NaN


In [15]:
for key in dict_all_coins.get( "DOGEUSDT").keys():
    print(key)

2021-04-14 20:00:00
2021-04-14 22:00:00
2021-04-15 00:00:00
2021-04-15 02:00:00
2021-04-15 04:00:00
2021-04-15 06:00:00
2021-04-15 08:00:00
2021-04-15 10:00:00
2021-04-15 12:00:00
2021-04-15 14:00:00
2021-04-15 16:00:00
2021-04-15 18:00:00
2021-04-15 20:00:00
2021-04-15 22:00:00
2021-04-16 00:00:00
2021-04-16 02:00:00
2021-04-16 04:00:00
2021-04-16 06:00:00
2021-04-16 08:00:00
2021-04-16 10:00:00
2021-04-16 12:00:00
2021-04-16 14:00:00
2021-04-16 16:00:00
2021-04-16 18:00:00
2021-04-16 20:00:00
2021-04-16 22:00:00
2021-04-17 00:00:00
2021-04-17 02:00:00
2021-04-17 04:00:00
2021-04-17 06:00:00
2021-04-17 08:00:00
2021-04-17 10:00:00
2021-04-17 12:00:00
2021-04-17 14:00:00
2021-04-17 16:00:00
2021-04-17 18:00:00
2021-04-17 20:00:00
2021-04-17 22:00:00
2021-04-18 00:00:00
2021-04-18 02:00:00
2021-04-18 04:00:00
2021-04-18 06:00:00
2021-04-18 08:00:00
2021-04-18 10:00:00
2021-04-18 12:00:00
2021-04-18 14:00:00
2021-04-18 16:00:00
2021-04-18 18:00:00
2021-04-18 20:00:00
2021-04-18 22:00:00
